In [1]:
import requests
import pickle
import pandas as pd
import json as js

In [10]:
game_data = []
year = '2019'
season_type = '02' 
max_game_ID = 1240

In [11]:
for i in range(0,max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)

In [12]:
game_data = game_data[1:]

In [9]:
game_data

KeyboardInterrupt: 

In [13]:
len(game_data)

1239

In [14]:
players_df = pd.DataFrame()

for i in game_data:
    tmp_players_df = pd.DataFrame(i['gameData']['players']).transpose()
    tmp_players_df = tmp_players_df.assign(gameId = i['gameData']['game']['pk'])
    tmp_players_df = tmp_players_df.assign(teamID = lambda x: x['currentTeam'].apply(lambda y: y['id']))
    tmp_players_df = tmp_players_df.assign(pPos = lambda x: x['primaryPosition'].apply(lambda y: y['name']))
    tmp_players_df = tmp_players_df.drop(columns=['currentTeam', 'primaryPosition'])
    players_df = players_df.append(tmp_players_df)
    
players_df.to_csv('players_df.csv', index=False)

players_df

TypeError: 'float' object is not subscriptable

In [7]:
teams_df = pd.DataFrame()

for i in game_data:
    tmp_teams_df = pd.DataFrame(i['gameData']['teams']).transpose()
    tmp_teams_df = tmp_teams_df.drop(columns=['venue','division','conference','franchise','officialSiteUrl','franchiseId'])
    teams_df = teams_df.append(tmp_teams_df)
    
teams_df.to_csv('teams_df.csv', index=False)

teams_df

KeyError: 'gameData'

In [15]:
games_df = pd.DataFrame()

for i in game_data:
    tmp_games_df = pd.DataFrame(i['gameData']['game'],index=[0])
    tmp_games_df = tmp_games_df.assign(startDate = i['gameData']['datetime']['dateTime'])
    tmp_games_df = tmp_games_df.assign(endDate = i['gameData']['datetime']['endDateTime'])
    tmp_games_df = tmp_games_df.assign(status = i['gameData']['status']['abstractGameState'])
    tmp_games_df = tmp_games_df.assign(teamAwayID = i['gameData']['teams']['away']['id'])
    tmp_games_df = tmp_games_df.assign(teamHomeID = i['gameData']['teams']['home']['id'])
    games_df = games_df.append(tmp_games_df)
    
games_df.to_csv('games_df.csv', index=False)
    
games_df

KeyError: 'endDateTime'

In [16]:
events_df = pd.DataFrame()

for i in game_data:
    tmp_events_df = pd.DataFrame(i['liveData']['plays']['allPlays'])
    tmp_events_df = tmp_events_df.assign(gameId = i['gameData']['game']['pk'])
    tmp_events_df = tmp_events_df.assign(eventId = lambda x: x['about'].apply(lambda y: y['eventIdx']))
    tmp_events_df = tmp_events_df.assign(eventType = lambda x: x['result'].apply(lambda y: y['event']))
    tmp_events_df = tmp_events_df.assign(eventTypeDesc = lambda x: x['result'].apply(lambda y: y['description']))
    tmp_events_df = tmp_events_df.assign(period = lambda x: x['about'].apply(lambda y: y['period']))
    tmp_events_df = tmp_events_df.assign(periodTime = lambda x: x['about'].apply(lambda y: y['periodTime']))
    tmp_events_df = tmp_events_df.assign(periodType = lambda x: x['about'].apply(lambda y: y['periodType']))
    tmp_events_df = tmp_events_df.assign(goalsHome = lambda x: x['about'].apply(lambda y: y['goals']['home']))
    tmp_events_df = tmp_events_df.assign(goalsAway = lambda x: x['about'].apply(lambda y: y['goals']['away']))
    tmp_events_df = tmp_events_df.assign(coX = lambda x: x['coordinates'].apply(lambda y: y['x'] if y else ''))
    tmp_events_df = tmp_events_df.assign(coY = lambda x: x['coordinates'].apply(lambda y: y['y'] if y else ''))

    result_player1Id = []
    result_player1Type = []

    for i in tmp_events_df['players']:
        try:
            result_player1Id.append(i[0]['player']['id'])
            result_player1Type.append(i[0]['playerType'])
        except IndexError:
            result_player1Id.append("")
            result_player1Type.append("")
        except TypeError:
            result_player1Id.append('')
            result_player1Type.append("")
        except ValueError:
            result_player1Id.append('')
            result_player1Type.append("")     

    result_player2Id = []
    result_player2Type = []

    for i in tmp_events_df['players']:
        try:
            result_player2Id.append(i[1]['player']['id'])
            result_player2Type.append(i[1]['playerType'])
        except IndexError:
            result_player2Id.append('')
            result_player2Type.append('')
        except TypeError:
            result_player2Id.append('')
            result_player2Type.append('')
        except ValueError:
            result_player2Id.append('')
            result_player2Type.append('')

    result_player3Id = []
    result_player3Type = []

    for i in tmp_events_df['players']:
        try:
            result_player3Id.append(i[2]['player']['id'])
            result_player3Type.append(i[2]['playerType'])
        except IndexError:
            result_player3Id.append('')
            result_player3Type.append('')
        except TypeError:
            result_player3Id.append('')
            result_player3Type.append('')
        except ValueError:
            result_player3Id.append('')
            result_player3Type.append('')

    result_player4Id = []
    result_player4Type = []

    for i in tmp_events_df['players']:
        try:
            result_player4Id.append(i[3]['player']['id'])
            result_player4Type.append(i[3]['playerType'])
        except IndexError:
            result_player4Id.append('')
            result_player4Type.append('')
        except TypeError:
            result_player4Id.append('')
            result_player4Type.append('')
        except ValueError:
            result_player4Id.append('')
            result_player4Type.append('')

    tmp_events_df = tmp_events_df.assign(player1Id = result_player1Id)
    tmp_events_df = tmp_events_df.assign(player1Type = result_player1Type)
    tmp_events_df = tmp_events_df.assign(player2Id = result_player2Id)
    tmp_events_df = tmp_events_df.assign(player2Type = result_player2Type)
    tmp_events_df = tmp_events_df.assign(player3Id = result_player3Id)
    tmp_events_df = tmp_events_df.assign(player3Type = result_player3Type)
    tmp_events_df = tmp_events_df.assign(player4Id = result_player4Id)
    tmp_events_df = tmp_events_df.assign(player4Type = result_player4Type)

    tmp_events_df = tmp_events_df.drop(columns=(['result','about','coordinates','players','team']))
                                      
    events_df = events_df.append(tmp_events_df)
    
events_df.to_csv('events_df.csv', index=False)

events_df

KeyError: 'about'